# Segmentation and Clustering of Neighborhoods in Toronto using Scraped Data\
_The following notebook is divided into three phases for purposes of Coursera's grading schema. 
1) In the first phase, I scrape a Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain data from the table therein that is used to create a table of postal codes and to transform the data into a pandas dataframe. 

2) In the second phase, I get this lat/long data for each neighborhood, using the Geocoder package, with a provision built in to ensure the data is pulled from an alternative CSV file if the goecoder package fails

3) Finally, having built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name,we can utilize Foursquare location data, using the the latitude and the longitude coordinates of each neighborhood in phase 2. I use this data to segment and cluster neighborhoods in Toronto.

Lets get started!

In [2]:
#First, we import the required packages and libraries. Doing this first saves time later. 
import numpy as np # So we can handle vectorized data

import pandas as pd # This is the primary library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# import the library we use to open URLs (to use in BeautifulSoup, our scraping package)
import urllib.request

# import the BeautifulSoup library so we can parse HTML and XML documents (and scrape the data from the Wikipedia table)
from bs4 import BeautifulSoup

import json # enables us to handle JSON files (the output of the scraped data)

from geopy.geocoders import Nominatim # Geopy enables geospatial analysis, this tool allows us convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

ModuleNotFoundError: No module named 'folium'

With this lengthy task out of the way, we can move on to creating the pandas dataframe we will use in later analysis. Thus begins phase 1 
## Phase 1

We start by scraping the table from wikipedia using BeautifulSoup and put it into a dataframe.

In [3]:
#Define source URL and use urllib package to get data, plugging that data into a page variable
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)

In [4]:
# Parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page, "lxml")
# Print using prettify in BeautifulSoup so we can see what we are working with
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XptG7wpAMNAAAUaw2@sAAAES","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":951325562,"wgRevisionId":951325562,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontario

Ok--we can see based on the above that we need to extract the table--note that the page contains quite a bit of code, but we see a 'wikitable sortable' class id which BeautifulSoup is able to quickly identify and extract. 

In [6]:
# Get title from HTML page
soup.title.string
# Find table and capture in a variable, display
postal_table = soup.find('table', class_='wikitable sortable')
postal_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postal code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park / Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor / Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park / Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern / Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3B
</td>
<td>North York
</td>
<td>Don Mills
</td></tr>
<tr>
<td>M4B
</td>
<td>Ea

Now that we have our table in an object, we can transform it into a pandas dataframe. However, before we can do this, we must loop through each row in the table, scraping the data into 3 empty lists (one for each column) which we will then plug into a pandas dataframe. The rows use the <td> tags and we want the contents of 'td' strings from within each of these (ie, the postal code ID, Borough, and neighborhood name.

In [7]:
# The code below creats 3 blank lists, one for each column in the table. Leaving them blank for simplicity's sake now, we will rename them in the next step. 
# Then, a for loop is created with a condition, ensuring that  
A = []
B = []
C = []
for row in postal_table.findAll('tr'):
        cells=row.findAll('td')
        if len(cells)==3:
            A.append(cells[0].find(text=True))
            B.append(cells[1].find(text=True))
            C.append(cells[2].find(text=True))

Ok, now we can create our pandas dataframe.

In [21]:
#we already imported pandas above, otherwise we would do so here
df = pd.DataFrame(A,columns=['Postal code'])
df['Borough']=B
df['Neighborhood']=C
#examine dataframe
df

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
7,M8A,Not assigned,
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge


Uh oh! we can see that several 'not assigned' boroughs have made it into our table. Let's get rid of those (note: unassigned neighborhoods are identical to the borough name--we will fix this as well.

In [22]:
#the following code should delete rows with 'not assigned' in the Boroughs column from the table
df = df[df.Borough != 'Not assigned']
df

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
7,M8A,Not assigned,
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge


Though I was unable to figure out why the above method didn't work,  the output demonstrates our dataframe still has extraneous data. This method is clearly not working, but we need a properly formatted dataframe in order to proceed with phase 2 . Let's try a different approach. Luckily, we can use read_html from pandas and instead use boolean indexing to recreate two dataframes, a workaround is both cleaner and that should enable us to proceed.

In [27]:
tdf1= pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

# Using boolean indexing, we can create one dataframe (tdf1) and then use it to create a second dataframe grouping neighborhoods who share postcodes into single rows, which may have created the above issue.
# I assumed the dataframe above might be messed up so we're starting with a fresh one here, but using a slightly different method (.loc) to work around the issue we encountered above
tdf1 = tdf1.loc[tdf1['Borough']!="Not assigned"]
tdf2=tdf1.groupby(['Postal code','Borough'],sort=False, as_index=False)['Neighborhood'].agg(','.join)
# if neighborhood isn't assigned, assume it is identical to borough.
tdf2.loc[tdf2['Neighborhood'] == "Not assigned", 'Neighborhood'] = tdf2['Borough']
tdf2

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,Malvern / Rouge
7,M3B,North York,Don Mills
8,M4B,East York,Parkview Hill / Woodbine Gardens
9,M5B,Downtown Toronto,"Garden District, Ryerson"


This methods works FAR better, and has given us what we need. Now that we have our tdf2 dataframe, we can proceed to the next step: using geocoder to find lat/long and appending them to the list with the appropriate postal code.

In [28]:
import geocoder # import geocoder

# initialize a variable to None
lat_lng_coords = None

# use a loop to get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(tdf2.Postal code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

SyntaxError: invalid syntax (<ipython-input-28-20527b673e58>, line 8)